<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1_v2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependecies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

In [ ]:
from scipy.fftpack import fft
from scipy.stats import spearmanr
from scipy.stats import pearsonr

# Import dataset

In [ ]:
"""Import Datasets"""
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)


# Data processing

In [ ]:
""" Data processing"""
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [ ]:
X.shape,Y.shape

((1867777, 3), (1867777,))

## low pass filter

In [ ]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]

## Frame preparation

In [ ]:
# frame preparation
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

In [ ]:
row_frame.shape, low_pass_frame.shape

((46693, 80, 3), (46693, 80, 3))

## Feature calculation

In [ ]:
#feature calculation
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0

In [ ]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])

    value = value / (frame_size / polling_rate)
    avc.append(value)


In [ ]:
max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))


In [ ]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))


In [ ]:
fft_x = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_x.append(fft(row_frame[i,:,0]))

fft_y = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_y.append(fft(row_frame[i,:,1]))

fft_z = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_z.append(fft(row_frame[i,:,2]))

energy_x = []
modulus_square = 0
for i in fft_x:
  for j in range(len(fft_x[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_x.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

energy_y = []
for i in fft_y:
  for j in range(len(fft_y[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_y.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

energy_z = []
for i in fft_z:
  for j in range(len(fft_z[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_z.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

'\ncorrelation = np.zeros((row_frame.shape[0],3))\nfor i in range(row_frame.shape[0]):\n  corr_xy, _ =  pearsonr(row_frame[i,:,0], row_frame[i,:,1])\n  corr_yz, _ =  pearsonr(row_frame[i,:,1], row_frame[i,:,2])\n  corr_xz, _ =  pearsonr(row_frame[i,:,0], row_frame[i,:,2])\n  correlation[i][0] = corr_xy\n  correlation[i][1] = corr_yz\n  correlation[i][2] = corr_xz\n'

In [ ]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 14))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2]
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]
    input_vector[i,11] = energy_x[i]
    input_vector[i,12] = energy_y[i]
    input_vector[i,13] = energy_z[i]
    #input_vector[i,9] = correlation[i,0]
    #input_vector[i,10] = correlation[i,1]
    #input_vector[i,11] = correlation[i,2]


for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+14] = angle[i][j]

In [ ]:
input_vector.shape, row_label.shape

((46693, 254), (46693,))

# Standarization

In [ ]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Trian Test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, row_label, test_size = 0.2, random_state = 0, stratify = row_label)

In [ ]:
X_train.shape, y_train.shape

((37354, 254), (37354,))

# Algorithm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, batch_size =64, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
584/584 [==============================] - 2s 3ms/step - loss: 1.0663 - accuracy: 0.6039 - val_loss: 1.0191 - val_accuracy: 0.6119
Epoch 2/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9752 - accuracy: 0.6227 - val_loss: 0.9714 - val_accuracy: 0.6205
Epoch 3/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9428 - accuracy: 0.6312 - val_loss: 0.9421 - val_accuracy: 0.6366
Epoch 4/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9187 - accuracy: 0.6365 - val_loss: 0.9197 - val_accuracy: 0.6432
Epoch 5/50
584/584 [==============================] - 2s 3ms/step - loss: 0.8994 - accuracy: 0.6420 - val_loss: 0.9227 - val_accuracy: 0.6397
Epoch 6/50
584/584 [==============================] - 2s 3ms/step - loss: 0.8866 - accuracy: 0.6472 - val_loss: 0.8903 - val_accuracy: 0.6472
Epoch 7/50
584/584 [==============================] - 2s 3ms/step - loss: 0.8730 - accuracy: 0.6488 - val_loss: 0.8838 - val_accuracy: 0.6516
Epoch 

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_, y_val)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,22,1,8,4,13,2,5,0,0,2,0,0
1,0,2,4,1,3,1,2,0,0,1,0,1
2,10,4,125,89,125,4,25,6,0,5,3,14
3,43,15,93,1057,547,63,186,8,15,9,13,50
4,52,15,269,821,4613,15,173,7,15,13,7,86
5,2,2,1,28,15,12,13,2,3,1,2,2
6,3,1,10,47,47,3,155,2,2,2,0,10
7,0,2,2,3,2,3,3,0,1,1,0,2
8,2,4,0,9,4,3,2,2,3,3,3,2
9,1,2,2,4,3,0,3,0,3,1,1,1


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_)

In [ ]:
per_class_accuracy = {}
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy[i] = accuracy

In [ ]:
per_class_accuracy

{0: 0.38596491228070173,
 1: 0.13333333333333333,
 2: 0.3048780487804878,
 3: 0.5035731300619343,
 4: 0.7579691094314821,
 5: 0.14457831325301204,
 6: 0.549645390070922,
 7: 0.0,
 8: 0.08108108108108109,
 9: 0.047619047619047616,
 10: 0.13333333333333333,
 11: 0.5243243243243243}

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
#y_pred = classifier.predict(X_val)

# Ensemble model

In [ ]:
y_pred_nn = model.predict(X_train)
y_pred_ = np.zeros((y_pred_nn.shape[0]))
for i in range(y_pred_nn.shape[0]):
    y_pred_[i] = np.argmax(y_pred_nn[i,:])
y_pred_nn = y_pred_

y_pred_rf = classifier.predict(X_train)

X_train_ensemble = np.concatenate((y_pred_nn, y_pred_rf),axis=0)

from sklearn.svm import SVC
blinder = SVC(kernel = 'linear', random_state = 0)
blinder.fit(X_train_ensemble, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
pd.DataFrame(cm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)